In [1]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import datetime as dt
import copy

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

- Đọc và chuẩn bị dữ liệu

In [2]:
#Đọc toàn bộ các file csv được xuất ra từ ami eod
full_stock_dict = {}
folder_path = 'D:\\t2m-project\\ami-data\\full_stock'
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        key = os.path.splitext(filename)[0]
        if len(key) == 3:
            full_stock_dict[key] = pd.read_csv(os.path.join(folder_path, filename)).sort_values('date', ascending=False).reset_index(drop=True)

for item, df in full_stock_dict.items():
    df = df[df['date'] >= 200101]
    df['date'] = pd.to_datetime(df['date'].astype(str), format='%y%m%d', errors='coerce')
    full_stock_dict[item] = df
    full_stock_dict[item]['ma20_volume'] = full_stock_dict[item]['volume'][::-1].rolling(window=20, min_periods=1).mean()[::-1]
    full_stock_dict[item]['value'] = full_stock_dict[item]['volume']*full_stock_dict[item]['close']
    full_stock_dict[item]['ma20_value'] = full_stock_dict[item]['value'][::-1].rolling(window=20, min_periods=1).mean()[::-1]

- Các hàm tính toán

In [3]:
# Hàm để tính ngày bắt đầu và kết thúc của một quý
def get_quarter_dates(year, quarter):
    if quarter == "q1":
        start_date = f"{year-1}-10-01"
        end_date = f"{year-1}-12-31"
    elif quarter == "q2":
        start_date = f"{year}-01-01"
        end_date = f"{year}-03-31"
    elif quarter == "q3":
        start_date = f"{year}-04-01"
        end_date = f"{year}-06-30"
    elif quarter == "q4":
        start_date = f"{year}-07-01"
        end_date = f"{year}-09-30"
    return start_date, end_date

# Hàm để xác định quý hiện tại
def get_current_quarter_year(name):
    now = datetime.now()
    year = now.year
    month = now.month
    if 1 <= month <= 3:
        quarter = "q1"
    elif 4 <= month <= 6:
        quarter = "q2"
    elif 7 <= month <= 9:
        quarter = "q3"
    else:
        quarter = "q4"
    if name == 'quarter':
        return quarter
    elif name == 'year':
        return year

- Tạo dict chứa các mốc thời gian từng quý

In [4]:
# Lấy ra quý và năm hiện tại
current_quarter = get_current_quarter_year('quarter')
current_year = int(get_current_quarter_year('year'))

# Tạo danh sách các quý và năm
quarters = ["q1", "q2", "q3", "q4"]
years = range(2020, current_year + 1)

# Tạo dict chứa các quý và mốc thời gian
period_date_dict = {}
for year in years:
    for quarter in quarters:
        if year == 2020: #Check điều kiện để không thêm quý 1 năm 2020
            if quarter != 'q1': 
                period_date_dict[f"{quarter}_{year}"] = get_quarter_dates(year, quarter)
        elif year == current_year: #Check điều kiện để không thêm các quý lớn hơn quý hiện tại
            if quarter <= current_quarter:
                period_date_dict[f"{quarter}_{year}"] = get_quarter_dates(year, quarter)
        else:
            period_date_dict[f"{quarter}_{year}"] = get_quarter_dates(year, quarter)

- Chuẩn bị danh sách cổ phiếu từng quý

In [5]:
#Tạo dict chứa danh cách cổ phiếu từng quý
period_stock_list_dict = {}
for period, date in period_date_dict.items():
    temp_dict = {k: v[(v['date'] >= date[0]) & (v['date'] <= date[1])] for k, v in copy.deepcopy(full_stock_dict).items()}
    temp_dict = [k for k, v in temp_dict.items() if (v['volume'].mean() >= 50000) & (v['ma20_volume'].min() >= 20000)&
                                                    (v['value'].mean() >= 500000) & (v['ma20_value'].min() >= 200000)]
    period_stock_list_dict[period] = temp_dict

#Tạo danh sách tất cả cổ phiếu quý trước
previous_full_stock_list = set()
for period in list(period_stock_list_dict.keys())[:-1]:
    stock_list = period_stock_list_dict[period]
    previous_full_stock_list.update(stock_list)

#Tạo danh tất cả sách cổ phiếu quý này
full_stock_list = set()
for period, stock_list in period_stock_list_dict.items():
    full_stock_list.update(stock_list)

#Thêm danh sách các cổ phiếu vào vị trí đầu tiên của dict
period_stock_list_dict = {**{'all_stock': list(full_stock_list)}, **period_stock_list_dict}

#In ra số lượng các cổ phiếu
for key, value in period_stock_list_dict.items():
    print(key, len(value))

#In ra các cổ phiếu mới làn đầu xuất hiện
new_stock_list = full_stock_list - previous_full_stock_list
print(f'Các cổ phiếu mới có: {new_stock_list}')
new_stock_df = pd.DataFrame(new_stock_list, columns=['new_stock_list'])

# Điền thêm giá trị NaN để đảm bảo các danh sách có cùng chiều dài
data = copy.deepcopy(period_stock_list_dict)
for key in data:
    if len(data[key]) < len(full_stock_list):
        data[key].extend([np.nan] * (len(full_stock_list) - len(data[key])))

# Tạo DataFrame
period_stock_list = pd.DataFrame(data)

all_stock 642
q2_2020 194
q3_2020 244
q4_2020 270
q1_2021 304
q2_2021 392
q3_2021 417
q4_2021 424
q1_2022 528
q2_2022 511
q3_2022 445
q4_2022 394
q1_2023 353
q2_2023 311
q3_2023 351
q4_2023 407
q1_2024 343
q2_2024 366
q3_2024 392
q4_2024 360
q1_2025 328
Các cổ phiếu mới có: {'HVA', 'TRC'}


- Lưu lại dữ liệu

In [6]:
# Hàm để tính ngày bắt đầu và kết thúc của một quý
def get_real_quarter_dates(year, quarter):
    if quarter == "q1":
        start_date = f"{year}-01-01"
        end_date = f"{year}-03-31"
    elif quarter == "q2":
        start_date = f"{year}-04-01"
        end_date = f"{year}-06-30"
    elif quarter == "q3":
        start_date = f"{year}-07-01"
        end_date = f"{year}-09-30"
    elif quarter == "q4":
        start_date = f"{year}-10-01"
        end_date = f"{year}-12-31"
    return [start_date, end_date]

# Lấy ra quý và năm hiện tại
current_quarter = get_current_quarter_year('quarter')
current_year = int(get_current_quarter_year('year'))

# Tạo danh sách các quý và năm
quarters = ["q1", "q2", "q3", "q4"]
years = range(2020, current_year + 1)

# Tạo dict chứa các quý và mốc thời gian
real_period_date_dict = {}
for year in years:
    for quarter in quarters:
        if year == 2020: #Check điều kiện để không thêm quý 1 năm 2020
            if quarter != 'q1': 
                real_period_date_dict[f"{quarter}_{year}"] = get_real_quarter_dates(year, quarter)
        elif year == current_year: #Check điều kiện để không thêm các quý lớn hơn quý hiện tại
            if quarter <= current_quarter:
                real_period_date_dict[f"{quarter}_{year}"] = get_real_quarter_dates(year, quarter)
        else:
            real_period_date_dict[f"{quarter}_{year}"] = get_real_quarter_dates(year, quarter)

for period, date in real_period_date_dict.items():
    date.append(len(period_stock_list_dict[period]))

period_map = pd.DataFrame.from_dict(real_period_date_dict, orient='index').reset_index()

In [7]:
with pd.ExcelWriter('period_stock_list.xlsx', engine='openpyxl') as writer:
    period_stock_list.to_excel(writer, sheet_name='period_stock_list', index=False)
    period_map.to_excel(writer, sheet_name='period_map', index=False)
    new_stock_df.to_excel(writer, sheet_name='new_stock_df', index=False)